# Prepare subset of gene trees and alignments for downstream analysis

Steps:

1. Make a list of all genes in the Gold-EggNOG overlap, and find GFF files for the corresponding genome assemblies
    - We need the nucleotide sequences for later analyses of selection on HGT
2. Make a list of all taxa and genes for which nucleotide sequences are available
3. Prepare a list of gene families (NOGs) that contain at least 4 of these taxa
3. Prepare sets of gene trees for HGT inference (at least 4 taxa each)
4. Select a subset of gene families (broadest possible, such that at least 95% of the selected taxa are present in each family) for estimating the genome tree
4. Extract these subsets of gene trees and alignments from the EggNOG database
5. Prune all gene trees to contain only the selected taxa and genes that have nucleotide sequences available

In [6]:
# suppress SyntaxWarnings in ete3 import using the warnings module
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [7]:
import os
import subprocess
import csv
import json
import multiprocessing as mp
from multiprocessing import Pool

import pandas as pd
import gffutils
import tqdm
import ete3
import numpy as np

In [8]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')
gold_e6_overlap_dir = os.path.join(data_dir, 'gold_e6_overlap')
e6_dir = os.path.join(data_dir, 'eggnog6')

filtered_dir = os.path.join(data_dir, 'filtered')
if not os.path.exists(filtered_dir):
    os.makedirs(filtered_dir)

In [9]:
# Define file paths
filtered_taxa_filepath = os.path.join(gold_e6_overlap_dir, 'e6_gold.filtered_taxa.txt')
nog2taxa_filepath = os.path.join(e6_dir, 'e6.og2seqs_and_species_bacteria.tsv')
filtered_genes_list_filepath = os.path.join(filtered_dir, 'genes.taxa_filtered.txt')

# Read in the gold-e6 overlap taxa list
filtered_taxa_df = pd.read_csv(filtered_taxa_filepath, sep='\t', names=['taxon_ID'], dtype={'taxon_ID': str})
filtered_taxa_set = set(filtered_taxa_df['taxon_ID'])

# Read the bacterial OGs to taxa mapping file in chunks
chunk_size = 10000
nog2taxa_chunks = pd.read_csv(nog2taxa_filepath, sep='\t', header=None, names=['level', 'nog', 'taxa_count', 'members_count', 'taxa', 'members'], chunksize=chunk_size)

unique_filtered_genes = set()

for chunk in nog2taxa_chunks:
    # Convert taxa and members columns into list of strings
    chunk['taxa'] = chunk['taxa'].str.split(',')
    chunk['members'] = chunk['members'].str.split(',')
    chunk['taxa_count'] = chunk['taxa_count'].astype(int)

    # Filter taxa to keep only those in the sampled taxa list
    chunk['filtered_taxa'] = chunk['taxa'].apply(lambda taxa_list: [taxon for taxon in taxa_list if taxon in filtered_taxa_set])
    chunk['filtered_taxa_count'] = chunk['filtered_taxa'].apply(len)

    # Drop NOGs that have less than 4 taxa
    chunk = chunk[chunk['filtered_taxa_count'] >= 4]

    # Filter members to keep only those that belong to the filtered taxa
    chunk['filtered_members'] = chunk['members'].apply(lambda members_list: [member for member in members_list if member.split('.')[0] in filtered_taxa_set])

    # Combine all filtered members into a single list and remove duplicates
    for members_list in chunk['filtered_members']:
        unique_filtered_genes.update(members_list)

# Write the unique filtered genes to a file
with open(filtered_genes_list_filepath, 'w') as file:
    for gene in unique_filtered_genes:
        file.write(f'{gene}\n')

# Print summary
print(f"Number of unique genes: {len(unique_filtered_genes)}")

Number of unique genes: 750755


## Find GFF files that contain the gene IDs for the Gold-EggNOG overlap

```bash
nohup python code/download_eggnog_ncbi_genome_assemblies.py -i data/filtered/genes.taxa_filtered.txt -o data/genome_data/ -c ~/ncbi_credentials.txt  > data/nohup_gff3_dload.log & disown
```


In [5]:
# make a list of all taxa for which we have available gene features
found_locus_tags_count_filepath = os.path.join(
    data_dir, 'genome_data/found_locus_tags_count.tsv')
found_locus_tags_count_df = pd.read_csv(found_locus_tags_count_filepath, sep='\t', header=0)
filtered_taxa_w_available_gff = found_locus_tags_count_df['taxon_id']
# write it to a file
filtered_taxa_w_available_gff_filepath = os.path.join(
    filtered_dir, 'taxa.overlap_filtered.gff_filtered.txt')
filtered_taxa_w_available_gff.to_csv(filtered_taxa_w_available_gff_filepath, index=False, header=False)
# display some
print(f"Number of taxa with available GFF files: {len(filtered_taxa_w_available_gff)}")
print(f"Example taxa with available GFF files: {filtered_taxa_w_available_gff.head().tolist()}")

Number of taxa with available GFF files: 159
Example taxa with available GFF files: [104087, 105219, 106654, 109790, 1221500]


In [6]:
# read in the list of locus tags found in the gff files
found_locus_tags_filepath = os.path.join(
    data_dir, 'genome_data/found_locus_tags.txt') # this file is created by the script 'code/download_eggnog_ncbi_genome_assemblies.py'
with open(found_locus_tags_filepath, 'r') as file:
    found_locus_tags = file.read().splitlines()

# make a list of all gene IDs searched for
gene_ids_filepath = os.path.join(filtered_dir, 'genes.taxa_filtered.txt')
gene_ids_df = pd.read_csv(gene_ids_filepath, header=None, names=['gene_id'])
gene_ids_df['taxon_id'] = gene_ids_df['gene_id'].str.split('.').str[0]
gene_ids_df['locus_tag'] = gene_ids_df['gene_id'].str.split('.').str[1]

# filter out locus tags that were not found in the gff files
found_gene_ids_df = gene_ids_df[gene_ids_df['locus_tag'].isin(found_locus_tags)]

# we now go through the gff files and extract the information for the genes we are interested in
gff_dir = os.path.join(data_dir, 'genome_data')
gff_files = [f for f in os.listdir(gff_dir) if f.endswith('.gff')]
gff_files = [os.path.join(gff_dir, f) for f in gff_files]
# function to extract gene information from a gff file and make a json file of locus tag features
def extract_gene_info_from_gff(gff_file, taxon_gene_ids_df):
    """
    Extract gene information from a gff file and return a dictionary of locus tag features.
    """
    # Create an in-memory database from the GFF file
    gff_db = gffutils.create_db(gff_file, ':memory:', merge_strategy='merge')
    gene_info = {}

    # extract taxon_id from the gff file name (first element of _ separated string)
    taxon_id = os.path.basename(gff_file).split('_')[0]

    # extract the accession ID from the gff db header
    # read the file line by line and extract the accession ID from the line starting with '#!genome-build-accession'
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#!genome-build-accession'):
                accession_id = line.split(':')[1].strip()
                break


    # Iterate over each feature of type 'gene' in the GFF database
    for feature in gff_db.features_of_type('gene'):
        # Get the locus tags and old locus tags from the feature attributes
        locus_tags = feature.attributes.get('locus_tag', [])
        old_locus_tags = feature.attributes.get('old_locus_tag', [])
        # Combine locus tags and old locus tags into a single set to avoid duplicates
        all_locus_tags = set(locus_tags + old_locus_tags)
        # Iterate over each locus tag in the combined set
        for locus_tag in all_locus_tags:
            # Check if the locus tag is in the list of found gene IDs
            if locus_tag in taxon_gene_ids_df['locus_tag'].values:
                # If yes, add the gene information to the gene_info dictionary
                gene_info[locus_tag] = {
                    'id': feature.id,  # Gene ID
                    'start': feature.start,  # Start position of the gene
                    'end': feature.end,  # End position of the gene
                    'strand': feature.strand,  # Strand information
                    'attributes': dict(feature.attributes),  # Feature attributes
                    'seqid': feature.seqid,  # Sequence ID
                    'accession_id': accession_id  # Genome accession ID
                }

    return taxon_id, gene_info

def extract_gene_info_from_gff_wrapper(args):
    return extract_gene_info_from_gff(*args)

# Initialize the all_gene_info dictionary
all_gene_info = {}
extract_gene_info_from_gff_args = [(gff_file, found_gene_ids_df[found_gene_ids_df['taxon_id'] == os.path.basename(gff_file).split('_')[0]]) for gff_file in gff_files]
# Extract gene information from all GFF files in parallel
num_gff_files = len(gff_files)
with Pool(mp.cpu_count()) as pool:
    results = pool.starmap(extract_gene_info_from_gff, extract_gene_info_from_gff_args)
    for taxon_id, gene_info in tqdm.tqdm(results, total=num_gff_files):
        all_gene_info[taxon_id] = gene_info

# Save the gene information to a JSON file
gene_info_json_filepath = os.path.join(filtered_dir, 'gene_features.json')
with open(gene_info_json_filepath, 'w') as json_file:
    json.dump(all_gene_info, json_file, indent=4)

print(f"Extracted gene information for {len(all_gene_info)} genes.")

100%|██████████| 159/159 [00:00<00:00, 220242.52it/s]


Extracted gene information for 159 genes.


# Prepare subset of EggNOG gene trees and alignments

In [7]:
# Create a set of found locus tags for faster lookup
found_locus_tags_set = set(found_locus_tags)

# Create a dictionary for faster lookup of gene_id to locus_tag
gene_id_to_locus_tag = {gene_id: locus_tag for gene_id, locus_tag in zip(
    gene_ids_df['gene_id'], gene_ids_df['locus_tag'])}

# Filter out genes that are not in the found_locus_tags set, from the nog2taxa_df members column
nog2taxa_df['filtered_members'] = nog2taxa_df['filtered_members'].apply(
    lambda members_list: [member for member in members_list if gene_id_to_locus_tag.get(member, '') in found_locus_tags_set])
nog2taxa_df['filtered_members_count'] = nog2taxa_df['filtered_members'].apply(
    len)

# similarly for the taxa column
nog2taxa_df['filtered_taxa'] = nog2taxa_df['filtered_members'].apply(
    lambda members_list: list(set([member.split('.')[0] for member in members_list])))
nog2taxa_df['filtered_taxa_count'] = nog2taxa_df['filtered_taxa'].apply(len)

# keep only nogs with at least 4 taxa
nog2taxa_df = nog2taxa_df[nog2taxa_df['filtered_taxa_count'] >= 4]
print(f"Number of NOGs after filtering, with at least 4 taxa and genes with available GFF files: {
      nog2taxa_df.shape[0]}")

# rearrange to have the counts before the lists
nog2taxa_df = nog2taxa_df[['level', 'nog', 'filtered_taxa_count',
                           'filtered_members_count', 'filtered_taxa', 'filtered_members']]

# make the lists as comma-separated strings
nog2taxa_df['filtered_taxa'] = nog2taxa_df['filtered_taxa'].apply(
    lambda taxa_list: ','.join(taxa_list))
nog2taxa_df['filtered_members'] = nog2taxa_df['filtered_members'].apply(
    lambda members_list: ','.join(members_list))

# to file
nog2taxa_filtered_filepath = os.path.join(
    filtered_dir, 'map.nog_taxa_members.tsv')
nog2taxa_df.to_csv(nog2taxa_filtered_filepath, sep='\t', index=False)
# make a list of the filtered nogs
filtered_nogs_filepath = os.path.join(filtered_dir, 'nogs.taxa_filtered.gff_filtered.txt')
nog2taxa_df['nog'].to_csv(filtered_nogs_filepath, index=False, header=False)

# prepare a NOG to members list map for all the NOGs that have only single copy genes (i.e. filtered_members_count == filtered_taxa_count)
single_copy_nogs2members_df = nog2taxa_df[nog2taxa_df['filtered_members_count'].astype(int) == nog2taxa_df['filtered_taxa_count'].astype(int)]
single_copy_nogs2members_df = single_copy_nogs2members_df[['nog', 'filtered_members']]
# write to file
single_copy_nogs2members_filepath = os.path.join(filtered_dir, 'map.nog_members_single_copy.tsv')
single_copy_nogs2members_df.to_csv(single_copy_nogs2members_filepath, sep='\t', index=False, header=False)
print(f"Number of single copy NOGs: {single_copy_nogs2members_df.shape[0]}")

Number of NOGs after filtering, with at least 4 taxa and genes with available GFF files: 8197
Number of single copy NOGs: 3718


Even if we don't have gene features for all genes in this list of NOGs, we know which taxa are present in them and we have gene trees estimated for them in EggNOG. We therefore use the original GOLD-EggNOG overlap list of NOGs and find the ones that are broadest in terms of representing all the taxa we have gene features for - to be used for genome tree inference. This means that the HGT inference we make later will be based on the set of gene trees that we have features for, but the genome tree will be based on a set of gene trees regardless of whether we have features for them or not.


In [5]:
# get NOGs that have all or almost all sampled taxa of GOLD-E6 overlap (for genome tree inference)
# read in original NOG2taxa file for the gold-e6 overlap
nog2taxa_filepath = os.path.join(e6_dir, 'e6.og2seqs_and_species_bacteria.tsv')
nog2taxa_df = pd.read_csv(nog2taxa_filepath, sep='\t', header=None, names=['level', 'nog', 'taxa_count', 'members_count', 'taxa', 'members'])
# remove all the NOGs not in nogs.taxa_filtered.gff_filtered.txt
filtered_nogs_filepath = os.path.join(filtered_dir, 'nogs.taxa_filtered.gff_filtered.txt')
with open(filtered_nogs_filepath, 'r') as file:
    filtered_nogs = file.read().splitlines()
# read in filtered taxa list
filtered_taxa_filepath = os.path.join(gold_e6_overlap_dir, 'e6_gold.filtered_taxa.txt')
filtered_taxa_df = pd.read_csv(filtered_taxa_filepath, sep='\t', names=['taxon_ID'], dtype={'taxon_ID': str})
filtered_taxa_list = filtered_taxa_df['taxon_ID'].tolist()
nog2taxa_df = nog2taxa_df[nog2taxa_df['nog'].isin(filtered_nogs)]
# keep only taxa that are in the filtered taxa list
nog2taxa_df['taxa'] = nog2taxa_df['taxa'].apply(lambda taxa_list: taxa_list.split(','))
nog2taxa_df['taxa'] = nog2taxa_df['taxa'].apply(lambda taxa_list: [taxon for taxon in taxa_list if taxon in filtered_taxa_list])
nog2taxa_df['taxa_count'] = nog2taxa_df['taxa'].apply(len)

# keep only NOGs that have at least 4 taxa
nog2taxa_df = nog2taxa_df[nog2taxa_df['taxa_count'] >= 4]
print(f"Number of NOGs with at least 4 of the filtered taxa: {nog2taxa_df.shape[0]}")

# first, get NOGs that have all or almost all sampled taxa of GOLD-E6 overlap. This is a small subset of NOGs
sampling_fraction = 0.95 # at least this fraction of the sampled taxa should be present in the NOG
sampling_taxa_count = int(sampling_fraction * len(filtered_taxa_list))
broad_distribution_nogs = nog2taxa_df[nog2taxa_df['taxa_count'] >= sampling_taxa_count].copy()
print(f"Number of NOGs where each of them have at least {sampling_fraction} of all filtered taxa ({
    sampling_taxa_count} out of {len(filtered_taxa_list)} taxa):", broad_distribution_nogs.shape[0])
# # sampling_fraction = 1.0
# broad_distribution_nogs = broad_distribution_nogs[broad_distribution_nogs['taxa_count'] == len(filtered_taxa_list)].copy()
# print(f"Number of NOGs where each of them have all filtered taxa ({len(filtered_taxa_list)} taxa):", broad_distribution_nogs.shape[0])

# write nogs list to file
broad_distribution_nogs_filepath = os.path.join(filtered_dir, 'nogs.broad_distribution.txt')
broad_distribution_nogs['nog'].to_csv(broad_distribution_nogs_filepath, index=False, header=False)

Number of NOGs with at least 4 of the filtered taxa: 8197
Number of NOGs where each of them have at least 0.95 of all filtered taxa (208 out of 219 taxa): 363


In [ ]:
# read in the nog2taxa file for NOGs with at least 4 taxa
nog2taxa_filepath = os.path.join(filtered_dir, 'map.nog_taxa_members.tsv')
nog2taxa_df = pd.read_csv(nog2taxa_filepath, sep='\t')
# find how many NOGs have single copy orthologs in all taxa
single_copy_nogs = nog2taxa_df[nog2taxa_df['filtered_members_count'] == len(filtered_taxa_list)].copy()

This was in the file mapping NOGs to constituent taxa or gene members. Now get a subset of trees and alignments that only contain the chosen NOGs:

In [9]:
ogs_chosen_path = f"{filtered_dir}/nogs.taxa_filtered.gff_filtered.txt"
all_trees_bacteria_path = f"{e6_dir}/e6.all_raw_trees_and_algs_bacteria.tsv"

# filepaths for the subset of the E6 trees/algs to be used in the analysis
trees_plus_algs_path = f"{filtered_dir}/map.nog_tree_alg.filtered.tsv"
trees_path = f"{filtered_dir}/map.nog_tree.filtered.tsv"
algs_path = f"{filtered_dir}/map.nog_alg.filtered.tsv"
nogs_with_trees_path = f"{filtered_dir}/nogs.filtered.with_trees.txt"

In [10]:
%%bash -s "$ogs_chosen_path" "$all_trees_bacteria_path" "$trees_plus_algs_path" "$trees_path" "$algs_path" "$nogs_with_trees_path"
cut -f3 $1 |tail -n +2 | rg -wFf - $2 > $3 # get list of chosen NOGs
# Separate trees in newick format from encoded alignments
cut -f1,3 $3 > $4 # extract trees from temp file
cut -f1,4 $3 > $5 # extract alignments from temp file
# rm $3
# Note that some of the trees don't exist in eggNOG for some of the NOGs that we chose.
# print the number of trees that are there in the trees_path file and the number of NOGs chosen
echo "Number of trees in the output file: $(wc -l $4)"
echo "Number of NOGs chosen (line count including header): $(wc -l $1)"
# print the NOG IDs that are there in 3rd column of chosen NOGs file but not in the 1st column of the trees_path file
# basically the difference between those two columns from the two files
echo "NOGs without trees:"
comm -23 <(cut -f3 $1 |tail -n +2 | sort) <(cut -f1 $4 |tail -n +2 | sort)

# make a list of NOGs that have trees
cut -f1 $4 |tail -n +2 > $6

Number of trees in the output file: 8186 /root/work/projects/hgt_ecosystem/data/filtered/map.nog_tree.filtered.tsv
Number of NOGs chosen (line count including header): 8197 /root/work/projects/hgt_ecosystem/data/filtered/nogs.taxa_filtered.gff_filtered.txt
NOGs without trees:
COG0236
COG0318
COG0438
COG0457
COG0515
COG0553
COG0642
COG0745
COG0784
COG1028
COG2202
COG2814
COG2931
COG3210
D9ST1


# Prune the gene trees


In [11]:
# first we prune the gene trees to be used for genome tree inference
# we keep only the taxa that are in the filtered taxa list

# Read the gene trees
gene_trees_path = f"{filtered_dir}/map.nog_tree.filtered.tsv"
gene_trees_df = pd.read_csv(gene_trees_path, header=None, index_col=None, sep='\t', names=['nog','tree'])

# read in the filtered nog2taxa file
nog2taxa_filtered_filepath = os.path.join(filtered_dir, 'map.nog_taxa_members.tsv')
nog2taxa_filtered_df = pd.read_csv(nog2taxa_filtered_filepath, sep='\t', header=0)

# we prune the gene trees to keep only the members that are in the filtered_members column of the nog2taxa file for each NOG
def prune_gene_tree_using_genes_list(args):
    nog, tree, genes_to_keep = args
    # read in the tree
    tree = ete3.Tree(tree, format=1)
    # prune the tree
    tree.prune(genes_to_keep)
    # check if the tree has at least 4 taxa
    taxa_in_tree = set([leaf.name.split('.')[0] for leaf in tree.get_leaves()])
    if len(taxa_in_tree) < 4:
        print(f"Tree {nog} has less than 4 taxa after pruning.")
        return None
    else:
        return nog, tree.write(format=1)
    
# make a three column df with NOG, tree and genes to keep
gene_trees_to_prune_df = pd.merge(gene_trees_df, nog2taxa_filtered_df[['nog', 'filtered_members']], how='inner', left_on='nog', right_on='nog')
gene_trees_to_prune_df['filtered_members'] = gene_trees_to_prune_df['filtered_members'].str.split(',')
gene_trees_to_prune_df = gene_trees_to_prune_df[['nog', 'tree', 'filtered_members']]
# convert the df to a list of tuples
gene_trees_to_prune_df = gene_trees_to_prune_df.values

# prune the gene trees in parallel using imap_unordered
gene_trees_pruned = {}
with Pool(mp.cpu_count()) as pool:
    for result in tqdm.tqdm(pool.imap_unordered(prune_gene_tree_using_genes_list, gene_trees_to_prune_df), total=len(gene_trees_to_prune_df)):
        if result is not None:
            nog, tree = result
            gene_trees_pruned[nog] = tree

# save the pruned gene trees to a file
gene_trees_pruned_filepath = os.path.join(filtered_dir, 'map.nog_tree.filtered.pruned.tsv')
with open(gene_trees_pruned_filepath, 'w') as file:
    file.write('\n'.join([f'{nog}\t{tree}' for nog, tree in gene_trees_pruned.items()]))
    file.write('\n')

# print the number of pruned gene trees
print(f"Number of pruned gene trees: {len(gene_trees_pruned)} out of {len(gene_trees_to_prune_df)}")

100%|██████████| 8182/8182 [00:37<00:00, 219.12it/s]


Number of pruned gene trees: 8182 out of 8182


In [12]:
%%bash -s "$filtered_dir" "$all_trees_bacteria_path" 
# now we prune the gene trees to be used for genome tree inference
# first we grep the trees for these broad distribution NOGs
rg -wFf $1/nogs.broad_distribution.txt $2 > $1/map.nog_tree_alg.broad_distribution.unpruned.tsv
# separate trees in newick format from encoded alignments
cut -f1,3 $1/map.nog_tree_alg.broad_distribution.unpruned.tsv > $1/map.nog_tree.broad_distribution.unpruned.tsv
cut -f1,4 $1/map.nog_tree_alg.broad_distribution.unpruned.tsv > $1/map.nog_alg.broad_distribution.unpruned.tsv

In [29]:
# read in the unpruned gene trees
unpruned_gene_trees_path = f'{filtered_dir}/map.nog_tree.broad_distribution.unpruned.tsv'
unpruned_gene_trees_df = pd.read_csv(unpruned_gene_trees_path, header=None, index_col=None, sep='\t', names=['nog','tree'])

# read in the filtered taxa list
filtered_taxa_filepath = os.path.join(filtered_dir, 'taxa.overlap_filtered.gff_filtered.txt')
with open(filtered_taxa_filepath, 'r') as file:
    filtered_taxa = file.read().splitlines()

def prune_gene_tree_using_taxa_list(args):
    nog, tree, taxa_to_keep = args
    # read in the tree
    tree = ete3.Tree(tree, format=1)
    # read in the leaves of the tree and make a list of leaves to keep
    # these are the leaves that have first element of the period separated leaf name in the taxa_to_keep list
    leaves_to_keep = [leaf for leaf in tree.get_leaves() if leaf.name.split('.')[0] in taxa_to_keep]
    # prune the tree
    tree.prune(leaves_to_keep)
    # check if the tree has at least 4 taxa
    taxa_in_tree = set([leaf.name.split('.')[0] for leaf in tree.get_leaves()])
    if len(taxa_in_tree) < 4:
        print(f"Tree {nog} has less than 4 taxa after pruning.")
        return None
    else:
        return nog, tree.write(format=1), taxa_to_keep, leaves_to_keep
    
# make a three column df with NOG, tree and taxa to keep
unpruned_gene_trees_to_prune_df = pd.DataFrame(unpruned_gene_trees_df)
unpruned_gene_trees_to_prune_df['taxa_to_keep'] = [filtered_taxa] * len(unpruned_gene_trees_to_prune_df)
unpruned_gene_trees_to_prune_df = unpruned_gene_trees_to_prune_df[['nog', 'tree', 'taxa_to_keep']]
# convert the df to a list of tuples
unpruned_gene_trees_to_prune_df = unpruned_gene_trees_to_prune_df.values

# prune the gene trees in parallel using imap_unordered
gene_trees_pruned = {}
broad_dist_nog_info = {}
with Pool(mp.cpu_count()) as pool:
    for result in tqdm.tqdm(pool.imap_unordered(prune_gene_tree_using_taxa_list, unpruned_gene_trees_to_prune_df), total=len(unpruned_gene_trees_to_prune_df)):
        if result is not None:
            nog, pruned_tree, taxa_to_keep, leaves_to_keep = result
            gene_trees_pruned[nog] = pruned_tree
            broad_dist_nog_info[nog] = {'taxa_count': len(taxa_to_keep), 'members_count': len(leaves_to_keep), 
                                        'taxa': ','.join(taxa_to_keep), 'members': ','.join([leaf.name for leaf in leaves_to_keep])}

# save the pruned gene trees to a file
gene_trees_pruned_filepath = os.path.join(filtered_dir, 'map.nog_tree.broad_distribution.pruned.tsv')
with open(gene_trees_pruned_filepath, 'w') as file:
    file.write('\n'.join([f'{nog}\t{tree}' for nog, tree in gene_trees_pruned.items()]))
    file.write('\n')

# write the second column (just the trees) to a file
broad_dist_gene_trees_pruned_filepath = os.path.join(filtered_dir, 'gene_trees.broad_distribution.pruned.nwk')
with open(broad_dist_gene_trees_pruned_filepath, 'w') as file:
    file.write('\n'.join([tree for tree in gene_trees_pruned.values()]) + '\n')

# save the broad distribution NOG info to a file
broad_dist_nog_info_filepath = os.path.join(filtered_dir, 'map.nog_taxa_members.broad_distribution.tsv')
broad_dist_nog_info_df = pd.DataFrame.from_dict(broad_dist_nog_info, orient='index')
broad_dist_nog_info_df.index.name = 'nog'
broad_dist_nog_info_df.reset_index(inplace=True)
broad_dist_nog_info_df.to_csv(broad_dist_nog_info_filepath, sep='\t', index=False)


# print the number of pruned gene trees
print(f"Number of pruned gene trees: {len(gene_trees_pruned)} out of {len(unpruned_gene_trees_to_prune_df)}")

# prepare a mapping file for astral, which maps genes to taxa
# this is a TSV file with two columns, the first column is a gene ID (e.g. 1234.ABC_789) and the second column is the taxon ID (e.g. 1234)
# we extract the members column from the broad distribution info df and explode to get a list of all gene IDs
broad_dist_gene_taxa_mapping_df = broad_dist_nog_info_df['members'].str.split(',', expand=False).explode().to_frame()
broad_dist_gene_taxa_mapping_df.columns = ['gene_id']
broad_dist_gene_taxa_mapping_df['taxon_id'] = broad_dist_gene_taxa_mapping_df['gene_id'].apply(lambda x: x.split('.')[0])
broad_dist_gene_taxa_mapping_df.to_csv(os.path.join(filtered_dir, 'map.gene_taxa.broad_distribution.txt'),
                                       sep=' ', index=False, header=False)

print(f"Number of genes in the broad distribution NOGs: {broad_dist_gene_taxa_mapping_df.shape[0]}")
print("Sample of the gene to taxa mapping:")
display(broad_dist_gene_taxa_mapping_df.head())

100%|██████████| 233/233 [00:17<00:00, 13.10it/s] 


Number of pruned gene trees: 233 out of 233
Number of genes in the broad distribution NOGs: 71906
Sample of the gene to taxa mapping:


,gene_id,taxon_id
0,546263.NELON_07490,546263
0,238.BBD35_03635,238
0,487316.BEN76_15855,487316
0,106654.B7L44_03485,106654
0,471.BUM88_16755,471


In [8]:
# Now prepare the alignments for IQTree.
# However, they have been compressed, in `map.nog_alg.broad_distribution.unpruned.tsv` so we need to decompress them first.
# Furthermore, these alignments contain all members that were present in the OG, but since we pruned the trees we need to prune the alignments as well.
# At the same time, we decide (at random) which paralog gets chosen as representative for the taxon ID, the others get removed.
import gzip
import base64
from io import StringIO
from Bio import AlignIO
from Bio.Nexus import Nexus
import glob
import shutil

alg_dir = os.path.join(filtered_dir, 'broad_distribution_algs')
if os.path.exists(alg_dir):
    print(f"Directory {alg_dir} already exists. Removing it.")
    shutil.rmtree(alg_dir)
os.makedirs(alg_dir)

filtered_taxa_w_available_gff_filepath = os.path.join(
    filtered_dir, 'taxa.overlap_filtered.gff_filtered.txt')
# read it in as a pd series
filtered_taxa_w_available_gff = pd.read_csv(filtered_taxa_w_available_gff_filepath, header=None, names=['taxon_id'], dtype=str)['taxon_id']

def convert_fasta_to_nexus(in_file: str, out_file: str, out_format: str = 'nexus'):
    # It works here, but not if I do it outside the function :)
    AlignIO.convert(in_file=in_file, in_format='fasta', out_file=out_file,
                    out_format=out_format, molecule_type='protein')
    return 0


def remove_paralogous_duplicate_taxa(alignments: list) -> set:
    """
    Removes paralogs from the alignment by choosing one member at random as representative for the taxon ID.
    Takes a list of SeqRecords as input.
    Returns alignment ids that are to be kept, the rest get dropped in another step.
    """
    taxa = {alignment.id.split('.')[0]: [] for alignment in alignments}
    for alignment in alignments:
        taxa[alignment.id.split('.')[0]].append(alignment.id)
    sample = {np.random.choice(taxa[gene]) for gene in taxa}
    return sample

############################################################################################################################################################################


# Read in aligments
algs = pd.read_csv(os.path.join(filtered_dir, 'map.nog_alg.broad_distribution.unpruned.tsv'),
                   header=None, names=['og', 'algs'], sep='\t')
print(f"Read in {algs.shape[0]} compressed alignments.")


def process_alignment(args):
    
    alg, taxa_set = args
    og_name = alg['og']
    compressed_algs = alg['algs']
    fasta_file_path = os.path.join(alg_dir, f'{og_name}.fasta')

    # Skip if the file already exists
    if os.path.exists(fasta_file_path):
        print(f"File {fasta_file_path} already exists. Skipping.")
        return

    # Decompress the file
    fasta = StringIO(gzip.decompress(base64.b64decode(compressed_algs)).decode())

    # Read the alignment
    alignment = AlignIO.read(fasta, format='fasta')

    # Retain only the taxa that are present in the pruned tree
    pruned_alignment = [record for record in alignment if record.id.split('.')[0] in taxa_set]

    # Remove duplicates of taxa if paralogs are present
    ids_to_keep = remove_paralogous_duplicate_taxa(pruned_alignment)
    pruned_alignment_without_duplicates = [record for record in pruned_alignment if record.id in ids_to_keep]

    # Write each alignment in a separate file per OG
    with open(fasta_file_path, 'w') as f:
        for record in pruned_alignment_without_duplicates:
            f.write(f'>{record.id.split(".")[0]}\n{record.seq}\n')

algs_list = algs.to_dict(orient='records')
filtered_taxa_w_available_gff_set = set(filtered_taxa_w_available_gff.astype(str).tolist())
print(f"Example taxa with available GFF files: {list(filtered_taxa_w_available_gff_set)[:5]}")
algs_args = [(alg, filtered_taxa_w_available_gff_set) for alg in algs_list]
print(f"Processing {len(algs_args)} alignments in parallel.")

# Use multiprocessing to process alignments in parallel
with Pool(mp.cpu_count() -2) as pool:
    list(tqdm.tqdm(pool.imap_unordered(process_alignment, algs_args), total=len(algs), desc='Pruning alignments'))


############################################################################################################################################################################

# Convert fasta to nexus if the nexus file doesn't exist or is older than the fasta file
nex_file_list = []
for fasta_file in tqdm.tqdm(glob.glob(os.path.join(os.path.expanduser(alg_dir), '*.fasta')), desc='Converting fasta to nexus'):
    # get basename without extension
    fasta_file_basename_root = os.path.splitext(os.path.basename(fasta_file))[0]
    nexus_file = os.path.join(alg_dir, f'{fasta_file_basename_root}.nex')

    # If the nexus file doesn't exist or is older than the fasta file, convert the fasta file
    if not os.path.exists(nexus_file) or os.path.getmtime(fasta_file) > os.path.getmtime(nexus_file):
        convert_fasta_to_nexus(fasta_file, nexus_file, 'nexus')
    else:
        print(f"File {nexus_file} is up-to-date. Skipping conversion.")

    nex_file_list.append(nexus_file)

# Concatenate Nexus files and write to a new file
nexi = [(fname, Nexus.Nexus(fname)) for fname in nex_file_list]
combined = Nexus.combine(nexi)
with open(os.path.join(filtered_dir, 'algs.filtered.concatenated.nex'), "w") as output_file:
    combined.write_nexus_data(filename=output_file, interleave=False)
print(f"Concatenated Nexus file written to {os.path.join(filtered_dir, 'algs.filtered.concatenated.nex')}")

Read in 233 compressed alignments.
Example taxa with available GFF files: ['587', '29542', '43767', '47917', '69370']
Processing 233 alignments in parallel.


Converting fasta to nexus: 100%|██████████| 233/233 [00:17<00:00, 12.98it/s]


Concatenated Nexus file written to /root/work/projects/hgt_ecosystem/data/filtered/algs.filtered.concatenated.nex


# Prepare Presence-Absence matrices 



## For NOGs

In [22]:
# make presence absence matrix for the full set of NOGs (not just the broad distribution ones)
# read in the filtered nog2taxa file as the file on members in the NOGs
nog2taxa_filtered_filepath = os.path.join(filtered_dir, 'map.nog_taxa_members.tsv')
nog2taxa_filtered_df = pd.read_csv(nog2taxa_filtered_filepath, sep='\t', header=0)

# break the taxa column into a list of taxa
nog2taxa_filtered_df['filtered_taxa'] = nog2taxa_filtered_df['filtered_taxa'].apply(
    lambda x: x.split(','))
nog2taxa_filtered_df['filtered_members'] = nog2taxa_filtered_df['filtered_members'].apply(
    lambda x: x.split(','))

# create a pa matrix for the full set of NOGs
# first create a list of all the taxa
taxa = list(
    set([item for sublist in nog2taxa_filtered_df['filtered_taxa'].tolist() for item in sublist]))
# sort the list
taxa.sort()

# create a dataframe with the NOGs as columns and the taxa as index
pa_df = pd.DataFrame(index=taxa, columns=nog2taxa_filtered_df['nog'], dtype=str).fillna('0')
for index, row in nog2taxa_filtered_df.iterrows():
    for taxon in row['filtered_taxa']:
        pa_df.loc[taxon, row['nog']] = str(int(len([gene for gene in row['filtered_members'] if gene.split('.')[0] == taxon])))

# display the size of the matrix/df
print("The size of the pa_df is: {}".format(pa_df.shape))

# TSV file for COUNT, with NOGs in index and taxa as columns
pa_df.T.to_csv(f"{filtered_dir}/pa_matrix.nogs.numerical.tsv", sep='\t')

# now binarize the matrix (i.e. if a NOG has at least one gene from a taxon, it gets a 1, otherwise 0)
pa_df = pa_df.map(lambda x: '1' if x != '0' else '0')

# Concatenate the columns into a single string per taxon and write out as FASTA file
fasta_df = pd.DataFrame(pa_df.astype(str).apply(''.join, axis=1), columns=['sequence'])
fasta_df.index = '>' + fasta_df.index.str.strip()
fasta_df_filepath = f"{filtered_dir}/pa_matrix.nogs.binary.fasta"
fasta_df.to_csv(f"{fasta_df_filepath}",
                sep='\n', header=False)

print(f"Presence-absence matrix written to {fasta_df_filepath}")

The size of the pa_df is: (159, 8197)
Presence-absence matrix written to /root/work/projects/hgt_ecosystem/data/filtered/pa_matrix.nogs.binary.fasta


## For Ecosystems

In [ ]:
import re

# read in the list of taxa after filtering for overlap and GFF files
taxa_filepath = os.path.join(filtered_dir, 'taxa.overlap_filtered.gff_filtered.txt')
with open(taxa_filepath, 'r') as file:
    filtered_taxa_list = file.read().splitlines()

# read in the tsv file for df mapping taxon IDs to ecosystem labels
taxon_ecosystem_map_filepath = os.path.join(data_dir, 'gold_e6_overlap/e6_gold.overlap_subset_df.tsv')
taxon_ecosystem_map_df = pd.read_csv(taxon_ecosystem_map_filepath, sep='\t', header=0)
print(f"Number of taxa in the taxon-ecosystem map that was read in: {taxon_ecosystem_map_df['taxon_ID'].nunique()}")

# filter the taxon_ecosystem_map_df to keep only the taxa that are in the filtered_taxa_list
taxon_ecosystem_map_df['taxon_ID'] = taxon_ecosystem_map_df['taxon_ID'].astype(str)
taxon_ecosystem_map_df = taxon_ecosystem_map_df[taxon_ecosystem_map_df['taxon_ID'].isin(filtered_taxa_list)]
print(f"Number of taxa in the taxon-ecosystem map after filtering for ones with available GFF files: {taxon_ecosystem_map_df['taxon_ID'].nunique()}")

# there are two kinds of pa-matrices to make: 
# one at the 'ORGANISM ECOSYSTEM TYPE' level and one at the 'ORGANISM ECOSYSTEM SUBTYPE' level (both are columns in the df)
taxon_ecosystem_type_df = taxon_ecosystem_map_df[['taxon_ID', 'ORGANISM ECOSYSTEM TYPE']]
taxon_ecosystem_subtype_df = taxon_ecosystem_map_df[['taxon_ID', 'ORGANISM ECOSYSTEM SUBTYPE']]

# take counts of taxon,ecosystem_type pairs and similarly for taxon,ecosystem_subtype pairs
taxon_ecosystem_type_counts_df = taxon_ecosystem_type_df.groupby(['taxon_ID', 'ORGANISM ECOSYSTEM TYPE']).size().reset_index(name='counts').astype(str)
taxon_ecosystem_subtype_counts_df = taxon_ecosystem_subtype_df.groupby(['taxon_ID', 'ORGANISM ECOSYSTEM SUBTYPE']).size().reset_index(name='counts').astype(str)

# pivot the dfs to get the pa matrices
taxon_ecosystem_type_pa_df = taxon_ecosystem_type_counts_df.pivot(columns='taxon_ID', index='ORGANISM ECOSYSTEM TYPE', values='counts').fillna('0').astype(str)
taxon_ecosystem_subtype_pa_df = taxon_ecosystem_subtype_counts_df.pivot(columns='taxon_ID', index='ORGANISM ECOSYSTEM SUBTYPE', values='counts').fillna('0').astype(str)

ecotype_index_names = taxon_ecosystem_type_pa_df.index.tolist()
# convert all spaces,hyphens,slashes to underscores in the index names
ecotype_index_names = {index:str(re.sub(r'[\s/-]', '_', index)) for index in ecotype_index_names}

# rename the index names
taxon_ecosystem_type_pa_df.rename(index=ecotype_index_names, inplace=True)

# similarly for the ecosystem subtype df
ecosubtype_index_names = taxon_ecosystem_subtype_pa_df.index.tolist()
ecosubtype_index_names = {index:str(re.sub(r'[\s/-]', '_', index)) for index in ecosubtype_index_names}
taxon_ecosystem_subtype_pa_df.rename(index=ecosubtype_index_names, inplace=True)

# from this ecosystem subtype df, drop the Unclassified row
taxon_ecosystem_subtype_pa_df.drop('Unclassified', inplace=True, axis=0)
# drop all columns that have only 0s
taxon_ecosystem_type_pa_df = taxon_ecosystem_type_pa_df.loc[(taxon_ecosystem_type_pa_df != '0').any(axis=1)]
taxon_ecosystem_subtype_pa_df = taxon_ecosystem_subtype_pa_df.loc[(taxon_ecosystem_subtype_pa_df != '0').any(axis=1)]

# write the pa matrices to files
taxon_ecosystem_type_pa_df.to_csv(os.path.join(filtered_dir, 'pa_matrix.ecosystem_type.numerical.tsv'), sep='\t')
taxon_ecosystem_subtype_pa_df.to_csv(os.path.join(filtered_dir, 'pa_matrix.ecosystem_subtype.numerical.tsv'), sep='\t')

# binarize the matrices
taxon_ecosystem_type_pa_df = taxon_ecosystem_type_pa_df.map(lambda x: '1' if x != '0' else '0')
taxon_ecosystem_subtype_pa_df = taxon_ecosystem_subtype_pa_df.map(lambda x: '1' if x != '0' else '0')

# Concatenate the rows into a single string per taxon and write out as FASTA file
taxon_ecosystem_type_fasta_df = pd.DataFrame(taxon_ecosystem_type_pa_df.astype(str).apply(''.join, axis=0), columns=['sequence'])
taxon_ecosystem_type_fasta_df.index = '>' + taxon_ecosystem_type_fasta_df.index.str.strip()
taxon_ecosystem_type_fasta_df.to_csv(f"{filtered_dir}/pa_matrix.ecosystem_type.binary.fasta", sep='\n', header=False)

taxon_ecosystem_subtype_fasta_df = pd.DataFrame(taxon_ecosystem_subtype_pa_df.astype(str).apply(''.join, axis=0), columns=['sequence'])
taxon_ecosystem_subtype_fasta_df.index = '>' + taxon_ecosystem_subtype_fasta_df.index.str.strip()
taxon_ecosystem_subtype_fasta_df.to_csv(f"{filtered_dir}/pa_matrix.ecosystem_subtype.binary.fasta", sep='\n', header=False)

print(f"Presence-absence matrices written out.")


Number of taxa in the taxon-ecosystem map that was read in: 219
Number of taxa in the taxon-ecosystem map after filtering for ones with available GFF files: 159
Presence-absence matrices written out.
